In [55]:
from transformers import pipeline
import torch
import pandas as pd
import re
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import folium
from folium.plugins import HeatMap
from itertools import permutations
import numpy as np

In [56]:
pipe = pipeline("token-classification", model="akdeniz27/bert-base-turkish-cased-ner",device=0)

c:\Users\Enes\anaconda3\envs\gputorch\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Example

In [57]:
text = "arkadaşımıza ulaşamıyoruz kahramanmaraş elbistan pınarbaşı mahallesi pınarbaşı caddesi cemre yapıcı kahramanmaraş deprem"

result = pipe(text)

print("Token ve Etiketler:")

location = ""
address_keywords = ["mahallesi", "caddesi", "sokak", "bulvarı", "köyü", "yolu", "mevkii", "mah.", "cad.", "sok.", "bul.", "köy.", "yol.", "mek.","apartman","apt"]

loc_tokens = []
for entity in result:
    token = entity['word']
    label = entity['entity']
    print(f"Token: {token}, Etiket: {label}")

    if "LOC" in label:  
        if token.startswith("##"):
            loc_tokens[-1] += token.lstrip("##")  
        else:
            loc_tokens.append(token)

words = text.split()  
for i, word in enumerate(words):
    if any(loc in word for loc in loc_tokens):
        location += word + " " 
    if any(keyword in word.lower() for keyword in address_keywords):
            location += word + " " 
loc_tokens
print("\nLocation:")
print(location.strip())

Token ve Etiketler:
Token: kahraman, Etiket: B-LOC
Token: ##maraş, Etiket: B-LOC
Token: elbis, Etiket: B-LOC
Token: ##tan, Etiket: B-LOC
Token: p, Etiket: B-LOC
Token: ##ınar, Etiket: B-LOC
Token: ##başı, Etiket: B-LOC
Token: p, Etiket: B-LOC
Token: ##ınar, Etiket: B-LOC
Token: ##başı, Etiket: B-LOC
Token: cem, Etiket: B-PER
Token: ##re, Etiket: B-PER
Token: kahraman, Etiket: B-LOC
Token: ##maraş, Etiket: B-LOC

Location:
kahramanmaraş elbistan pınarbaşı mahallesi pınarbaşı caddesi kahramanmaraş


In [58]:
text = "çocuk enkazın altında ai̇leşi̇ i̇ki̇ gündür acil müdahale edi̇lmesi̇ni̇ bekliyor adres akasya mahallesi şükrü balcı caddesi sara apartmanı hatay antakya iletişim deprem sondakikadeprem acil acildeprem"

result = pipe(text)

print("Token ve Etiketler:")

location = ""

loc_tokens = []
for entity in result:
    token = entity['word']
    label = entity['entity']
    print(f"Token: {token}, Etiket: {label}")

    # LOC etiketli tokenları topla
    if 'LOC' in label:  
            if not token.startswith('##'):  
                if location:  
                    location += " " 
                location += token
            else:
                location += token.lstrip('##')  


print("\nLocation:")
print(location.strip())


Token ve Etiketler:
Token: ak, Etiket: B-LOC
Token: ##asy, Etiket: B-LOC
Token: ##a, Etiket: B-LOC
Token: bal, Etiket: I-PER
Token: ##cı, Etiket: I-PER
Token: sar, Etiket: B-LOC
Token: ##a, Etiket: B-LOC
Token: hata, Etiket: B-LOC
Token: ##y, Etiket: B-LOC
Token: ant, Etiket: B-LOC
Token: ##ak, Etiket: B-LOC
Token: ##ya, Etiket: B-LOC

Location:
akasya sara hatay antakya


In [59]:
geolocator = Nominatim(user_agent="geoapi")

# Adres olarak tam metni belirtiyoruz
address = "hatay esenlik mahallesi"

# Adresi enlem ve boylama çevir
location = geolocator.geocode(address)

if location:
    print(f"Adres: {address}")
    print(f"Enlem: {location.latitude}, Boylam: {location.longitude}")
else:
    print(f"Adres bulunamadı: {address}")

Adres: hatay esenlik mahallesi
Enlem: 36.2059571, Boylam: 36.1478466


# Get Location 

In [60]:
df = pd.read_csv("../results/TweetAnalyzeResult.csv")
df.head()

,timestamp,username,handle,tweet_date,content,content_ment_link,content_wo_punct,content_wo_removed_english,content_wo_normalize,content_wo_tokenize,content_wo_stop,content_wo_lemmatized,content_no_rare_words
0,2024-11-03 19:38:25,Orhan Şevik,orhan_sevik,"Feb 8, 2023",@haluklevent\n @oguzhanugur\n hepinizden alla...,\n \n hepinizden allah razı olsun iyiki varsı...,hepinizden allah razı olsun iyiki varsını...,hepinizden allah razı olsun iyiki varsınız deprem,hepinizden allah razı olsun iyi ki varsınız de...,"['hepinizden', 'allah', 'razı', 'olsun', 'iyi'...","['hepinizden', 'razı', 'varsınız', 'deprem']","['hep', 'razı', 'var', 'deprem']","['hep', 'razı', 'var', 'deprem']"
1,2024-11-03 19:38:25,nursena,nurssxx_,"Feb 8, 2023",allahım nolur sen onlara dayanma gücü ver #deprem,allahım nolur sen onlara dayanma gücü ver #deprem,allahım nolur sen onlara dayanma gücü ver deprem,allahım nolur onlara dayanma gücü ver deprem,allahım ne olur onlara dayanma gücü ver deprem,"['allahım', 'ne', 'olur', 'onlara', 'dayanma',...","['onlara', 'dayanma', 'gücü', 'ver', 'deprem']","['o', 'dayan', 'güc', 'ver', 'deprem']","['o', 'dayan', 'güc', 'ver', 'deprem']"
2,2024-11-03 19:38:25,Elif ŞEKER,55ellllllif,"Feb 8, 2023",replying to \n@haluklevent\n @danlabilic\n and...,replying to \n\n \n and \n_harun\nadıyamanda a...,replying to and harun adıyamanda ali taş...,harun adıyamanda ali taşı mahallesi sokak hic...,harun adıyamanda ali taşı mahallesi sokak hicr...,"['harun', 'adıyamanda', 'ali', 'taşı', 'mahall...","['harun', 'adıyamanda', 'ali', 'taşı', 'mahall...","['haru', 'adıyaman', 'ali', 'taşı', 'mahalle',...","['adıyaman', 'ali', 'taşı', 'mahalle', 'sokak'..."
3,2024-11-03 19:38:25,Yusuf,yusufaltuns,"Feb 8, 2023",tuğba södekoğlu kovulsun \n@showtv\n #deprem,tuğba södekoğlu kovulsun \n\n #deprem,tuğba södekoğlu kovulsun deprem,tuğba södekoğlu kovulsun deprem,tuğba sodekoğlu kovulsun deprem,"['tuğba', 'sodekoğlu', 'kovulsun', 'deprem']","['tuğba', 'sodekoğlu', 'kovulsun', 'deprem']","['tuğba', 'sodekoğlu', 'kov', 'deprem']",['deprem']
4,2024-11-03 19:38:25,𝐎̈𝐳𝐠𝐮̈𝐫 𝐑𝐚𝐧,dryghtn2,"Feb 8, 2023",arkadaşimiza ulaşamiyoruz\nkahramanmaraş elbis...,arkadaşimiza ulaşamiyoruz\nkahramanmaraş elbis...,arkadaşimiza ulaşamiyoruz kahramanmaraş elbist...,arkadaşimiza ulaşamiyoruz kahramanmaraş elbist...,arkadaşımıza ulaşamıyoruz kahramanmaraş elbist...,"['arkadaşımıza', 'ulaşamıyoruz', 'kahramanmara...","['arkadaşımıza', 'ulaşamıyoruz', 'kahramanmara...","['arkadaş', 'ulaş', 'kahramanmaraş', 'elbistan...","['arkadaş', 'ulaş', 'kahramanmaraş', 'elbistan..."


In [61]:
def capitalize(text):
    words = text.split()
    capitalized_words = [word.capitalize() for word in words]
    capitalized_text = ' '.join(capitalized_words)
    return capitalized_text

df['content_wo_tokenize_capitalize'] = df['content_wo_normalize'].apply(capitalize)

In [62]:
def get_location_from_text(text):
    if not text.strip():
        return None 
    result = pipe(text) 
    location = "" 
    
    for entity in result:
        token = entity['word']
        label = entity['entity']

        if 'LOC' in label:  
            if not token.startswith('##'):  
                if location:  
                    location += " " 
                location += token
            else:
                location += token.lstrip('##')  
   

    return location.strip() if location else None

df['location'] = df['content_wo_tokenize_capitalize'].apply(get_location_from_text)

In [63]:
def remove_duplicates(text):
    if not text:
        return text
    words = text.split()
    unique_words = list(set(words))
    unique_words.sort(key=lambda x: words.index(x))  
    return ' '.join(unique_words)

df['location_unique'] = df['location'].apply(remove_duplicates)

# Special Address

In [64]:
df['tokenized_location'] = df['location_unique'].apply(lambda x: x.split() if pd.notnull(x) else [])

In [65]:
geolocator = Nominatim(user_agent="geoapi")

def get_coordinates(address):
    if not address or pd.isna(address):
        return None, None
    try:
        location = geolocator.geocode(address, timeout=10)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except GeocoderTimedOut:
        return None, None

In [66]:
get_coordinates("Hatay Arsuz gözcüler ")

(36.388756, 35.916489)

#### All combination

In [67]:
turkiye_adres = pd.read_excel("../data/ililcemahalle.xlsx")

turkiye_adres["il"] = turkiye_adres["il"].str.strip()
turkiye_adres["ilçe"] = turkiye_adres["ilçe"].str.strip()
turkiye_adres["semt_bucak_belde"] = turkiye_adres["semt_bucak_belde"].str.strip()
turkiye_adres["Mahalle"] = turkiye_adres["Mahalle"].str.strip()

iller = set(turkiye_adres["il"].str.lower().unique())
ilceler = set(turkiye_adres["ilçe"].str.lower().unique())
semtler = set(turkiye_adres["semt_bucak_belde"].str.lower().unique())
mahalleler = set(turkiye_adres["Mahalle"].str.lower().unique())

In [68]:
def contains_in_set(token, keyword_set):
    return [keyword for keyword in keyword_set if keyword in token]

def clean_address(address):
    return [word for word in address if 'hal' not in word.lower()]

def build_address_from_tokens(tokens):
    if not isinstance(tokens, list) or not tokens or len(tokens) == 1:
        return None  
    
    found_parts = []  
    tokens = [token.lower() for token in tokens]  

    for token in tokens:
        if (matched_parts := contains_in_set(token, iller)):
            if matched_parts[0] not in found_parts:
                found_parts.append(matched_parts[0])  
        elif (matched_parts := contains_in_set(token, ilceler)):
            if matched_parts[0] not in found_parts:
                found_parts.append(matched_parts[0])

    found_parts = clean_address(found_parts)
    
    return " ".join(found_parts) if found_parts else None

In [69]:
df["address"] = df["tokenized_location"].apply(build_address_from_tokens)

In [70]:
def update_coordinates(row):
    if pd.notna(row['address']) and len(row['address'].split()) > 1: 
        return get_coordinates(row['address'])
    return None, None  


df["latitude"], df["longitude"] = zip(*df.apply(update_coordinates, axis=1))

## City

In [71]:
sehirler = [
    "Adana", "Adıyaman", "Afyon", "Ağrı", "Amasya", "Ankara", "Antalya", "Artvin",
    "Aydın", "Balıkesir", "Bilecik", "Bingöl", "Bitlis", "Bolu", "Burdur", "Bursa", "Çanakkale",
    "Çankırı", "Çorum", "Denizli", "Diyarbakır", "Edirne", "Elazığ", "Erzincan", "Erzurum", 
    "Eskişehir", "Gaziantep", "Giresun", "Gümüşhane", "Hakkari", "Hatay", "Isparta", "Mersin",
    "İstanbul", "İzmir", "Kars", "Kastamonu", "Kayseri", "Kırklareli", "Kırşehir", "Kocaeli",
    "Konya", "Kütahya", "Malatya", "Manisa", "Kahramanmaraş", "Mardin", "Muğla", "Muş", 
    "Nevşehir", "Niğde", "Ordu", "Rize", "Sakarya", "Samsun", "Siirt", "Sinop", "Sivas", 
    "Tekirdağ", "Tokat", "Trabzon", "Tunceli", "Şanlıurfa", "Uşak", "Van", "Yozgat", 
    "Zonguldak", "Aksaray", "Bayburt", "Karaman", "Kırıkkale", "Batman", "Şırnak", 
    "Bartın", "Ardahan", "Iğdır", "Yalova", "Karabük", "Kilis", "Osmaniye", "Düzce"
]


### Find City

In [72]:
def extract_cities(tokens, city_list):
    if not tokens:  
        return []
    matches = []
    for city in city_list:
        for token in tokens:
            if re.search(r'\b' + re.escape(city) + r'\b', token, re.IGNORECASE): 
                matches.append(city)
    return list(set(matches))  

In [73]:
df['city'] = df['tokenized_location'].apply(lambda x: extract_cities(x, sehirler))

### Get latitude & longitude

In [74]:
def update_city_coordinates(row):
    if isinstance(row['city'], list) and len(row['city']) > 0 and pd.notna(row['city'][0]):
        return get_coordinates(row['city'][0])
    return None, None

In [75]:
df["latitude_city"], df["longitude_city"] = zip(*df.apply(update_city_coordinates, axis=1))

In [77]:
df.to_csv('../results/LocationResult.csv', index=False)